In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os

In [2]:
# -------------------- Helper Functions --------------------

# def add_prediction_column(base_df, new_df, new_col_name):
#     # Ensure 'actual' in new_df is rounded and converted to int
#     new_df = new_df.copy()
#     new_df['actual'] = new_df['actual'].round().astype(int)

#     grouped_base = base_df.groupby('geocode')
#     grouped_new = new_df.groupby('geocode')
#     aligned_preds = []

#     for geocode, base_group in grouped_base:
#         new_group = grouped_new.get_group(geocode)
#         assert len(base_group) == len(new_group), f"Row count mismatch for geocode {geocode}"

#         base_actuals = base_group['actual'].values
#         new_actuals = new_group['actual'].values
#         if not np.array_equal(base_actuals, new_actuals):
#             mismatches = np.where(base_actuals != new_actuals)[0]
#             for idx in mismatches:
#                 week = base_group.iloc[idx]['week']
#                 base_val = base_actuals[idx]
#                 new_val = new_actuals[idx]
#                 print(f"Geocode {geocode}, row {idx}, week {week}: base actual = {base_val}, new actual = {new_val}")
#             raise AssertionError(f"Actual mismatch for geocode {geocode}")

#         aligned_preds.extend(new_group[new_col_name].values)

#     base_df[new_col_name] = aligned_preds
#     return base_df

def add_prediction_column(base_df, new_df, new_col_name):
    new_df = new_df.copy()
    new_df['actual'] = new_df['actual'].round().astype(int)

    # Merge on geocode and week
    merged = base_df[['geocode', 'week', 'actual']].merge(
        new_df[['geocode', 'week', 'actual', new_col_name]],
        on=['geocode', 'week'],
        how='left',
        suffixes=('_base', '_new')
    )

    # Validate actual values (with tolerance)
    actual_base = merged['actual_base'].values
    actual_new = merged['actual_new'].values
    diffs = np.abs(actual_base - actual_new)
    mismatches = np.where(diffs > 1)[0]

    if len(mismatches) > 0:
        for idx in mismatches:
            row = merged.iloc[idx]
            print(f"Geocode {row['geocode']}, week {row['week']}: base actual = {row['actual_base']}, new actual = {row['actual_new']}")
        raise AssertionError("Actual mismatch (tolerance exceeded)")

    # Merge prediction column to base_df
    base_df = base_df.merge(
        new_df[['geocode', 'week', new_col_name]],
        on=['geocode', 'week'],
        how='left'
    )

    return base_df


def drop_first_n_rows_per_geocode(df, n=8):
    return df.groupby('geocode', group_keys=False).apply(lambda group: group.iloc[n:]).reset_index(drop=True)

def load_and_prepare_model_preds(years, model_name, file_pattern, column_name):
    dfs = []
    for year in years:
        path = file_pattern.format(year)
        df = pd.read_csv(path)
        df = df.rename(columns={column_name: model_name})
        df['actual'] = df['actual'].round().astype(int)
        df['year'] = year
        
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

def load_and_prepare_model_preds_handle_missing(years, model_name, file_pattern, column_name):
    dfs = []
    for year in years:
        path = file_pattern.format(year)
        df = pd.read_csv(path)

        # Convert the prediction column to numeric, coercing errors (empty to NaN)
        df[column_name] = pd.to_numeric(df[column_name], errors='coerce')

        # Rename the prediction column to model_name
        df = df.rename(columns={column_name: model_name})

        # Round 'actual' to int only if no missing values, otherwise keep as float (or drop rows with missing actual later)
        df['actual'] = pd.to_numeric(df['actual'], errors='coerce').round()

        df['year'] = year
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


In [3]:
# -------------------- Load and Combine Base Model Predictions (2019–2021) --------------------

years = [2019, 2020, 2021]
xgboost_df = load_and_prepare_model_preds(years, 'XGBoost', "../base_predictions/xgboost_preds_{}.csv", 'xgboost_pred')
svr_df = load_and_prepare_model_preds(years, 'SVR', "../base_predictions/svr_preds_{}.csv", 'svr_pred')
rf_df = load_and_prepare_model_preds(years, 'RF', "../base_predictions/rf_preds_{}.csv", 'rf_pred')
catboost_df = load_and_prepare_model_preds(years, 'CatBoost', "../base_predictions/catboost_preds_{}.csv", 'catboost_pred')
lightgbm_df = load_and_prepare_model_preds(years, 'LightGBM', "../base_predictions/lightgbm_preds_{}.csv", 'lightgbm_predict')
lstm_df = load_and_prepare_model_preds_handle_missing(years, 'LSTM', "../base_predictions/lstm_preds_{}.csv", 'lstm_pred')


for df in [xgboost_df, svr_df, rf_df, catboost_df, lightgbm_df, lstm_df]:
  print(df.columns)

Index(['geocode', 'week', 'XGBoost', 'actual', 'year'], dtype='object')
Index(['geocode', 'week', 'SVR', 'actual', 'year'], dtype='object')
Index(['geocode', 'week', 'RF', 'actual', 'year'], dtype='object')
Index(['geocode', 'week', 'CatBoost', 'actual', 'year'], dtype='object')
Index(['geocode', 'week', 'LightGBM', 'actual', 'year'], dtype='object')
Index(['geocode', 'LSTM', 'actual', 'year'], dtype='object')


In [4]:
# -------------------- Assemble Training Set --------------------

ensemble_df = xgboost_df[['geocode', 'week', 'actual', 'year', 'XGBoost']].copy()
ensemble_df['actual'] = ensemble_df['actual'].round().astype(int)

ensemble_df = add_prediction_column(ensemble_df, svr_df, 'SVR')
ensemble_df = add_prediction_column(ensemble_df, rf_df, 'RF')
ensemble_df = add_prediction_column(ensemble_df, catboost_df, 'CatBoost')

# Sort lightgbm_df and lstm_df by geocode and year
lightgbm_df = lightgbm_df.sort_values(by=['geocode', 'year', 'week']).reset_index(drop=True)
lstm_df = lstm_df.sort_values(by=['geocode', 'year']).reset_index(drop=True)

# We'll add 'LSTM' column to lightgbm_df matching rows group-wise
lstm_col = 'LSTM'

result_groups = []
for (geocode, year), lb_group in lightgbm_df.groupby(['geocode', 'year']):
    lstm_group = lstm_df[(lstm_df['geocode'] == geocode) & (lstm_df['year'] == year)]
    if len(lb_group) != len(lstm_group):
        raise ValueError(f"Row count mismatch for geocode {geocode}, year {year}")
    lb_group = lb_group.copy()
    lb_group[lstm_col] = lstm_group[lstm_col].values
    result_groups.append(lb_group)

lightgbm_df = pd.concat(result_groups, ignore_index=True)

ensemble_df = add_prediction_column(ensemble_df, lightgbm_df, 'LightGBM')
ensemble_df = ensemble_df.merge(
    lightgbm_df[['geocode', 'week', 'LSTM']], 
    on=['geocode', 'week'], 
    how='left'
) 

# Sort to get desired order: geocode-wise, then week-wise
ensemble_df = ensemble_df.sort_values(by=['geocode', 'week']).reset_index(drop=True)

features = ['XGBoost', 'SVR', 'RF', 'CatBoost', 'LightGBM', 'LSTM']

# Check and remove rows with NaNs in prediction columns
print("\nMissing values per feature:")
print(ensemble_df[features].isnull().sum())

# Drop rows with any missing predictions
ensemble_df = ensemble_df.dropna(subset=features)

X = ensemble_df[features].values
y = ensemble_df['actual'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)



Missing values per feature:
XGBoost       0
SVR           0
RF            0
CatBoost      0
LightGBM      0
LSTM        364
dtype: int64


In [5]:
# # GridSearchCV for hyperparameter tuning
# param_grid = {
#     'n_estimators': [50, 100, 150],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'max_depth': [2, 3, 4]
# }

# gbr = GradientBoostingRegressor(random_state=42)
# grid_search = GridSearchCV(gbr, param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1)
# grid_search.fit(X_train, y_train)

# # Best model
# meta_model = grid_search.best_estimator_
# print(f"\nBest Hyperparameters: {grid_search.best_params_}")

# y_pred = meta_model.predict(X_val)

# mae = mean_absolute_error(y_val, y_pred)
# rmse = np.sqrt(mean_squared_error(y_val, y_pred))

# print(f"\nEvaluation on Test Data:")
# print(f"Mean Absolute Error (MAE): {mae:.4f}")
# print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

In [6]:
# # -------------------- Train and Evaluate Meta-Models --------------------

# meta_models = {
#     'Ridge': Ridge(),
#     'Lasso': Lasso(),
#     'LinearRegression': LinearRegression()
# }

# param_grids = {
#     'Ridge': {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
#     'Lasso': {'alpha': [0.001, 0.01, 0.1, 1.0]},
#     'LinearRegression': {}  # No params
# }

# meta_model = None
# lowest_rmse = float('inf')

# for name, model in meta_models.items():
#     print(f"\n--- Training {name} ---")
#     grid_search = GridSearchCV(model, param_grids[name], scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
#     grid_search.fit(X_train, y_train)

#     best_model = grid_search.best_estimator_
#     y_pred = best_model.predict(X_val)
#     mae = mean_absolute_error(y_val, y_pred)
#     rmse = np.sqrt(mean_squared_error(y_val, y_pred))

#     print(f"{name} — MAE: {mae:.4f}, RMSE: {rmse:.4f}, Best Params: {grid_search.best_params_}")

#     if rmse < lowest_rmse:
#         lowest_rmse = rmse
#         meta_model = best_model

# print(f"\nBest Meta-Model: {meta_model.__class__.__name__} with RMSE: {lowest_rmse:.4f}")

In [7]:
meta_models = {
    # 'Ridge': Ridge(),
    # 'Lasso': Lasso(),
    # 'LinearRegression': LinearRegression(),
    
    # SVR with StandardScaler inside a pipeline
    'SVR': Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR())
    ])
}

param_grids = {
    'Ridge': {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
    'Lasso': {'alpha': [0.001, 0.01, 0.1, 1.0]},
    'LinearRegression': {},  # No hyperparameters

    # Use prefix 'svr__' because SVR is inside the pipeline under name 'svr'
    # 'SVR': {
    #     'svr__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    #     'svr__C': [0.1, 1, 10, 100],
    #     'svr__gamma': ['scale', 'auto'],
    #     'svr__degree': [2, 3],  # Only used by 'poly' kernel
    #     'svr__epsilon': [0.1, 0.2]
    # }

        'SVR': {
        'svr__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'svr__C': [0.1, 1],
        'svr__gamma': ['scale', 'auto'],
        'svr__degree': [2, 3],  # Only used by 'poly' kernel
        'svr__epsilon': [0.001, 0.01, 0.1, 1.0]
    }
}

meta_model = None
lowest_rmse = float('inf')

for name, model in meta_models.items():
    print(f"\n--- Training {name} ---")
    grid_search = GridSearchCV(model, param_grids[name], scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    print(f"{name} — MAE: {mae:.4f}, RMSE: {rmse:.4f}, Best Params: {grid_search.best_params_}")

    if rmse < lowest_rmse:
        lowest_rmse = rmse
        meta_model = best_model

print(f"\nBest Meta-Model: {meta_model.__class__.__name__} with RMSE: {lowest_rmse:.4f}")



--- Training SVR ---
Fitting 5 folds for each of 128 candidates, totalling 640 fits
SVR — MAE: 2.3619, RMSE: 15.0964, Best Params: {'svr__C': 1, 'svr__degree': 2, 'svr__epsilon': 1.0, 'svr__gamma': 'scale', 'svr__kernel': 'linear'}

Best Meta-Model: Pipeline with RMSE: 15.0964


In [8]:
# -------------------- Load 2022 Predictions for Testing --------------------
def load_test_df(file_path, model_name, pred_col):
    df = pd.read_csv(file_path)
    df = df.rename(columns={pred_col: model_name})
    if 'actual' in df.columns:
        df['actual'] = pd.to_numeric(df['actual'], errors='coerce').round()
    return df

def load_lstm_test_df(file_path):
    df = pd.read_csv(file_path)
    df = df.rename(columns={'lstm_pred': 'LSTM'})
    df['actual'] = pd.to_numeric(df['actual'], errors='coerce').round().astype('Int64')
    return df


xgboost_test_df = load_test_df("../base_predictions/xgboost_preds_2022.csv", 'XGBoost', 'xgboost_pred')
svr_test_df = load_test_df("../base_predictions/svr_preds_2022.csv", 'SVR', 'svr_pred')
rf_test_df = load_test_df("../base_predictions/rf_preds_2022.csv", 'RF', 'rf_pred')
catboost_test_df = load_test_df("../base_predictions/catboost_preds_2022.csv", 'CatBoost', 'catboost_pred')
lightgbm_test_df = load_test_df("../base_predictions/lightgbm_preds_2022.csv", 'LightGBM', 'lightgbm_predict')
lstm_test_df = load_lstm_test_df("../base_predictions/lstm_preds_2022.csv")

# Reset index to ensure identical row ordering
lightgbm_test_df = lightgbm_test_df.sort_values(by=['geocode', 'week']).reset_index(drop=True)
lstm_test_df = lstm_test_df.reset_index(drop=True)

# Check alignment
if len(lightgbm_test_df) != len(lstm_test_df):
    raise ValueError("Row count mismatch between LightGBM and LSTM test sets")

# Add LSTM column
lightgbm_test_df['LSTM'] = lstm_test_df['LSTM'].values


# Start with XGBoost + ground truth
ensemble_eval_df = xgboost_test_df[['geocode', 'week', 'actual', 'XGBoost']].copy()

# Add other predictions
ensemble_eval_df = add_prediction_column(ensemble_eval_df, svr_test_df, 'SVR')
ensemble_eval_df = add_prediction_column(ensemble_eval_df, rf_test_df, 'RF')
ensemble_eval_df = add_prediction_column(ensemble_eval_df, catboost_test_df, 'CatBoost')
ensemble_eval_df = add_prediction_column(ensemble_eval_df, lightgbm_test_df, 'LightGBM')

# Finally add LSTM predictions from LightGBM df
ensemble_eval_df['LSTM'] = lightgbm_test_df['LSTM']


In [9]:
# -------------------- Generate and Save Ensemble Predictions --------------------

X_2022 = ensemble_eval_df[features].values
ensemble_eval_df['ensemble_pred'] = meta_model.predict(X_2022)

# Set negative predictions to 0 (dengue cases can't be negative)
ensemble_eval_df['ensemble_pred'] = ensemble_eval_df['ensemble_pred'].clip(lower=0)

ensemble_eval_df.to_csv("../ensemble_predictions/final_ensemble_predictions_2022.csv", index=False)

# Evaluation
if 'actual' in ensemble_eval_df.columns:
    y_true = ensemble_eval_df['actual']
    y_pred = ensemble_eval_df['ensemble_pred']

    mae_2022 = mean_absolute_error(y_true, y_pred)
    rmse_2022 = np.sqrt(mean_squared_error(y_true, y_pred))
    r2_2022 = r2_score(y_true, y_pred)

    print(f"\nEvaluation on Hold-out Set (2022):")
    print(f"MAE  = {mae_2022:.4f}")
    print(f"RMSE = {rmse_2022:.4f}")
    print(f"R²   = {r2_2022:.4f}")


Evaluation on Hold-out Set (2022):
MAE  = 2.3411
RMSE = 8.6770
R²   = 0.8391


In [10]:
# -------------------- Plotting Results per Geocode --------------------

output_dir = "../ensemble_predictions/plots_2022"
os.makedirs(output_dir, exist_ok=True)

model_cols = ['XGBoost', 'SVR', 'RF', 'CatBoost', 'LightGBM', 'LSTM']

for geocode in ensemble_eval_df['geocode'].unique():
    df_geo = ensemble_eval_df[ensemble_eval_df['geocode'] == geocode].sort_values(by='week')
    plt.figure(figsize=(14, 6))

    plt.plot(df_geo['week'], df_geo['actual'], label='Actual', color='black', linewidth=2.5, linestyle='--')
    for model in model_cols:
        plt.plot(df_geo['week'], df_geo[model], label=model, linewidth=1.2)
    plt.plot(df_geo['week'], df_geo['ensemble_pred'], label='Ensemble', color='blue', linewidth=2.5)

    plt.title(f"Geocode {geocode} — 2022 Predictions vs Actual")
    plt.xlabel("Week")
    plt.ylabel("Dengue Cases")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"geocode_{geocode}.png"))
    plt.close()